# 🚀 Model Deployment Challenge

**Welcome to the MLOps Challenge!** You've trained and optimized your model in **04.2**. Now it's time to **deploy it to production** and make it available for real-world predictions!

---

## 🎯 Your Mission

Deploy your ML model using **two production strategies**:

1. **Batch Inference (Spark UDF)** - Process thousands of turbines daily
2. **Real-Time API (REST Endpoint)** - Instant predictions for live sensor data

---

## 📊 The Challenge

**Scenario:** Your energy company needs predictions in two ways:

**🔄 Batch Processing:**
- Run daily analysis on all 10,000 turbines
- Generate maintenance reports
- Store predictions in data warehouse
- **Use Case:** Overnight batch jobs, historical analysis

**⚡ Real-Time API:**
- IoT sensors send live data
- Instant prediction (< 1 second)
- Trigger alerts for critical failures
- **Use Case:** Mobile apps, monitoring dashboards, alerting systems

---

## 🏆 Challenge Levels

### **Level 1: Batch Inference with Spark UDF (30 min)**
- Load registered model from Unity Catalog
- Create Spark User-Defined Function (UDF)
- Apply predictions to entire dataset
- Save results as table

### **Level 2: REST API Deployment (45 min)**
- Create Model Serving Endpoint
- Configure scale-to-zero for cost optimization
- Wait for endpoint deployment (~15 min)
- Test API with sample data

### **Level 3: Advanced Testing & Integration (30 min)**
- Build robust API client function
- Test different input formats
- Handle errors gracefully
- Compare batch vs real-time performance

---

## 📝 What You'll Learn

✅ **Spark UDFs** - Integrate ML into SQL/DataFrames  
✅ **Model Serving** - Production-grade REST APIs  
✅ **Scale-to-Zero** - Cost-effective deployment  
✅ **Batch vs Real-Time** - Choose the right strategy  
✅ **API Testing** - POST requests and error handling  
✅ **MLOps Best Practices** - Versioning, monitoring, lineage  

---

Let's deploy! 🚀

---

## 📦 Setup: Install Libraries

**What you need to do:**
Install required packages for model deployment.

**💡 Note:** After installation, Python kernel will restart automatically.

In [ ]:
# RUN THIS CELL - No changes needed
%pip install --quiet databricks-sdk==0.40.0 mlflow==2.22.0
dbutils.library.restartPython()

In [ ]:
# RUN THIS CELL - No changes needed
%run ../_resources/00-setup $reset_all_data=false

---

## 📥 Task 1: Import Libraries & Configure MLflow

**What you need to do:**
1. Import necessary libraries for deployment
2. Configure MLflow to use Unity Catalog

**Libraries you'll need:**
- **mlflow:** Model loading and serving
- **mlflow.deployments:** Deployment client for endpoints
- **requests:** HTTP API calls
- **json:** JSON data formatting
- **pandas & numpy:** Data manipulation

**💡 Hint:** Same setup as 04.2!

In [ ]:
# YOUR CODE HERE: Import libraries
# Hint: import numpy as np
#       import pandas as pd
#       import mlflow
#       from mlflow.models import infer_signature
#       from mlflow import MlflowClient
#       from mlflow.deployments import get_deploy_client
#       import os
#       import requests
#       import json
#       import time

In [ ]:
# YOUR CODE HERE: Configure MLflow to use Unity Catalog
# Hint: mlflow.set_registry_uri('databricks-uc')

### ✅ Success Criteria:
- All libraries imported without errors
- MLflow configured for Unity Catalog

---

# 🎯 LEVEL 1: Batch Inference with Spark UDF (30 min)

**Goal:** Deploy model for batch predictions on large datasets

---

## 📊 Background: What is a Spark UDF?

**User-Defined Function (UDF):** Custom function that can be used in Spark SQL and DataFrames

**Why use UDFs for ML?**
- ✅ Process millions of rows in parallel
- ✅ Use SQL for predictions (accessible to analysts)
- ✅ Seamlessly integrate ML into data pipelines
- ✅ Results saved as tables for downstream use

**How it works:**
```python
# 1. Load model as Spark UDF
predict_udf = mlflow.pyfunc.spark_udf(spark, model_uri)

# 2. Apply to DataFrame
df.withColumn('prediction', predict_udf(col1, col2, ...))

# 3. Use in SQL
SELECT predict_udf(col1, col2) FROM table
```

---

## 📝 Task 2: Load Model from Unity Catalog

**What you need to do:**
1. Define model name (same as in 04.2)
2. Create Spark UDF from registered model
3. Register UDF for use in SQL queries

**💡 Important:**
- Use `@prod` alias to load production model
- `env_manager='virtualenv'` ensures consistent environment
- ⏱️ **First run takes 15+ minutes** to build virtual environment

**Model URI format:** `models:/{catalog}.{db}.{model_name}@prod`

In [ ]:
# YOUR CODE HERE: Define model name
# Hint: model_name = "turbine_maintenance"

In [ ]:
# YOUR CODE HERE: Create Spark UDF from model
# This loads the model in a virtual environment (can take 15+ minutes!)

# Hint:
# predict_maintenance = mlflow.pyfunc.spark_udf(
#     spark, 
#     f"models:/{catalog}.{db}.{model_name}@prod",
#     result_type="float",  # Output type
#     env_manager='virtualenv'
# )

print("⏳ Building virtual environment... This may take 15+ minutes on first run.")

In [ ]:
# YOUR CODE HERE: Register UDF for SQL use
# This allows you to call predict_maintenance() in SQL queries

# Hint: spark.udf.register("predict_maintenance", predict_maintenance)

### ✅ Success Criteria:
- UDF created successfully (may take 15+ min)
- UDF registered for SQL use
- No errors during loading

---

## 📊 Task 3: Inspect Model Input Schema

**What you need to do:**
1. Get list of input columns the model expects
2. Display input schema
3. Verify column names match your feature table

**💡 Why?** Ensures you pass the correct features in the right order!

In [ ]:
# YOUR CODE HERE: Get input column names
# Hint: columns = predict_maintenance.metadata.get_input_schema().input_names()

In [ ]:
# YOUR CODE HERE: Display columns
# Print the list to see what features the model expects

In [ ]:
# YOUR CODE HERE: Display full input schema
# Hint: predict_maintenance.metadata.get_input_schema()

### 🤔 Reflection Question:
**What columns does the model expect?**
```
[Your answer: Should be 7 features - avg_energy, std_sensor_A through F]
```

---

## 🔄 Task 4: Apply UDF to DataFrame (Batch Predictions)

**What you need to do:**
1. Load `turbine_hourly_features` table
2. Apply UDF to add prediction column
3. Display results
4. Save predictions as new table: `turbine_hourly_predictions`

**💡 Structure:**
```python
df = spark.table('turbine_hourly_features')
df_with_pred = df.withColumn('prediction', predict_maintenance(*columns))
df_with_pred.write.mode('overwrite').saveAsTable('turbine_hourly_predictions')
```

In [ ]:
# YOUR CODE HERE: Apply UDF to DataFrame

# Step 1: Load feature table
# Hint: spark.table('turbine_hourly_features')

# Step 2: Add prediction column using UDF
# Hint: .withColumn("predict_turbine_maintenance", predict_maintenance(*columns))

# Step 3: Display results
# Hint: display(batch_pred_df)

# Step 4: Save as table
# Hint: batch_pred_df.write.mode("overwrite").saveAsTable("turbine_hourly_predictions")

### ✅ Success Criteria:
- Predictions added as new column
- Predictions are numeric (0, 1, 2 for different failure types)
- Table `turbine_hourly_predictions` created successfully
- All rows have predictions (no NULLs)

---

## 📝 Task 5: Use UDF in SQL Query

**What you need to do:**
Write a SQL query that:
1. Uses the `predict_maintenance()` UDF
2. Selects `turbine_id` and prediction
3. Passes all required sensor features
4. Limits to 10 rows

**💡 This shows how analysts can use your ML model directly in SQL!**

In [ ]:
# YOUR CODE HERE: Write SQL query using UDF

%sql
-- Example structure:
-- SELECT turbine_id, 
--        predict_maintenance(
--            avg_energy, 
--            std_sensor_A, 
--            std_sensor_B, 
--            std_sensor_C, 
--            std_sensor_D, 
--            std_sensor_E, 
--            std_sensor_F
--        ) as prediction 
-- FROM turbine_hourly_features
-- LIMIT 10

### 🤔 Reflection Questions:

**1. What are the advantages of UDFs for batch inference?**
```
[Your answer: Think about scale, integration with existing pipelines, SQL accessibility]
```

**2. When would you use batch inference vs real-time?**
```
[Your answer: Consider latency requirements, data volume, use cases]
```

**3. How many turbines did you score? How long did it take?**
```
[Your answer: Check row count and execution time]
```

---

# 🚀 LEVEL 2: REST API Deployment (45 min)

**Goal:** Deploy model as a REST API endpoint for real-time predictions

---

## 📡 Background: Why REST API Endpoints?

**REST API Serving:** Model hosted as web service

**Benefits:**
- ⚡ **Real-time:** Sub-second predictions
- 🌍 **Accessible:** Call from any application (mobile, web, IoT)
- 🔒 **Secure:** Token-based authentication
- 💰 **Cost-effective:** Scale-to-zero when idle

**Use Cases:**
- Mobile app for field technicians
- Live monitoring dashboard
- Alerting system for critical failures
- Integration with external systems

---

## 🔧 Task 6: Prepare Endpoint Configuration

**What you need to do:**
1. Display endpoint name (defined in setup)
2. Initialize deployment client
3. Check if endpoint already exists
4. Delete existing endpoint if found (for clean deployment)

**💡 Note:** `MODEL_SERVING_ENDPOINT_NAME` is set in the setup script

In [ ]:
# YOUR CODE HERE: Display endpoint name
# Hint: print(MODEL_SERVING_ENDPOINT_NAME)

In [ ]:
# YOUR CODE HERE: Delete existing endpoint if it exists
# This ensures a clean deployment

# Hint:
# client = get_deploy_client("databricks")
# 
# for endpoint in client.list_endpoints():
#     if endpoint['name'] == MODEL_SERVING_ENDPOINT_NAME:
#         print(f"Deleting existing endpoint: {MODEL_SERVING_ENDPOINT_NAME}")
#         client.delete_endpoint(MODEL_SERVING_ENDPOINT_NAME)
#         print("Endpoint deleted successfully")

### ✅ Success Criteria:
- Endpoint name displayed
- Deployment client initialized
- Old endpoint deleted (if existed)

---

## 🚀 Task 7: Create Serving Endpoint

**What you need to do:**
1. Create endpoint configuration
2. Deploy model from Unity Catalog
3. Configure workload size and scale-to-zero
4. Wait for endpoint to be ready (~15 minutes)

**⏱️ Important:** Endpoint creation takes **15+ minutes** - be patient!

**Configuration Options:**
- `workload_size`: "Small" (sufficient for testing)
- `scale_to_zero_enabled`: True (cost optimization)
- `entity_version`: Get latest version from registry

**💡 Scale-to-Zero:** Endpoint shuts down after 15 min of inactivity, restarts automatically on request

In [ ]:
# YOUR CODE HERE: Create serving endpoint
# This spins up a container to run the model for inference

# Step 1: Initialize deployment client
# client = get_deploy_client("databricks")

# Step 2: Create endpoint
# try:
#     endpoint = client.create_endpoint(
#         name=MODEL_SERVING_ENDPOINT_NAME,
#         config={
#             "served_entities": [
#                 {
#                     "name": "iot-maintenance-serving-endpoint",
#                     "entity_name": f"{catalog}.{db}.{model_name}",
#                     "entity_version": get_last_model_version(f"{catalog}.{db}.{model_name}"),
#                     "workload_size": "Small",
#                     "scale_to_zero_enabled": True
#                 }
#             ]
#         }
#     )
#     print(f"✅ Endpoint {MODEL_SERVING_ENDPOINT_NAME} creation started!")
# except Exception as e:
#     if "already exists" in str(e):
#         print(f"Endpoint {MODEL_SERVING_ENDPOINT_NAME} already exists.")
#     else:
#         raise e

print("⏳ Waiting for endpoint to be ready... (15+ minutes)")

In [ ]:
# YOUR CODE HERE: Wait for endpoint to be ready
# This checks status every 10 seconds until ready

# Hint:
# while client.get_endpoint(MODEL_SERVING_ENDPOINT_NAME)['state']['config_update'] == 'IN_PROGRESS':
#     print("⏳ Still deploying...")
#     time.sleep(10)
# 
# if client.get_endpoint(MODEL_SERVING_ENDPOINT_NAME)['state']['ready'] != 'READY':
#     print(f"❌ Endpoint {MODEL_SERVING_ENDPOINT_NAME} creation failed.")
# else:
#     print(f"✅ Endpoint {MODEL_SERVING_ENDPOINT_NAME} is READY!")

### ✅ Success Criteria:
- Endpoint creation started successfully
- Status check loop running
- Final status: READY
- ⏱️ Total time: ~15 minutes

### 🤔 Reflection Question:
**Why does endpoint deployment take so long?**
```
[Your answer: Think about virtual environment creation, dependency installation, container setup]
```

---

# 🧪 LEVEL 3: Advanced Testing & Integration (30 min)

**Goal:** Test endpoint and build robust API client

---

## 🔐 Task 8: Get API Credentials

**What you need to do:**
1. Get Databricks workspace URL
2. Get API token for authentication

**💡 Security:** Never hardcode tokens! Always get them from secure context.

In [ ]:
# YOUR CODE HERE: Get API credentials

# Hint:
# API_ROOT = f"https://{dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().value()}/"
# API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

# Verify (don't print full token for security!)
# print(f"API Root: {API_ROOT}")
# print(f"Token loaded: {API_TOKEN is not None}")

---

## 🛠️ Task 9: Build API Client Function

**What you need to do:**
Create a `score_model()` function that:
1. Formats input data correctly (DataFrame or dict)
2. Sends POST request to endpoint
3. Handles authentication
4. Returns predictions or raises error

**💡 The function must handle two input formats:**
- **Pandas DataFrame:** Convert to `dataframe_split` format
- **Dict/Array:** Convert to TensorFlow Serving format

**Structure:**
```python
def score_model(dataset):
    # 1. Construct endpoint URL
    # 2. Set headers (auth + content-type)
    # 3. Format data based on type
    # 4. Send POST request
    # 5. Handle response
```

In [ ]:
# YOUR CODE HERE: Create helper function for TensorFlow format

def create_tf_serving_json(data):
    """
    Prepares input data in JSON format expected by TensorFlow Serving.
    Converts data to nested dict under 'inputs' key.
    """
    # Hint: Check if data is dict, convert values to lists
    # If not dict, convert directly to list
    # Return: {'inputs': {...}}
    
    pass  # Remove and write your code

In [ ]:
# YOUR CODE HERE: Create score_model function

def score_model(dataset):
    """
    Send prediction request to model serving endpoint.
    
    Args:
        dataset: Pandas DataFrame or dict with features
        
    Returns:
        Predictions as JSON
    """
    
    # Step 1: Construct endpoint URL
    # Hint: url = f'{API_ROOT}/serving-endpoints/{MODEL_SERVING_ENDPOINT_NAME}/invocations'
    
    # Step 2: Set headers
    # Hint: headers = {'Authorization': f'Bearer {API_TOKEN}', 'Content-Type': 'application/json'}
    
    # Step 3: Format data based on type
    # Hint: If DataFrame → {'dataframe_split': dataset.to_dict(orient='split')}
    #       Else → create_tf_serving_json(dataset)
    
    # Step 4: Convert to JSON
    # Hint: data_json = json.dumps(ds_dict, allow_nan=True)
    
    # Step 5: Send POST request
    # Hint: response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    
    # Step 6: Handle response
    # Hint: Check response.status_code, raise exception if not 200
    
    pass  # Remove and write your code

### ✅ Success Criteria:
- Function handles both DataFrames and dicts
- Proper error handling for failed requests
- Returns JSON response with predictions

---

## 🧪 Task 10: Test API with Sample Data

**What you need to do:**
1. Load sample data (5 rows from feature table)
2. Select only required columns
3. Convert to pandas
4. Call `score_model()` function
5. Display predictions

**Required columns:**
- `avg_energy`, `std_sensor_A`, `std_sensor_B`, `std_sensor_C`, `std_sensor_D`, `std_sensor_E`, `std_sensor_F`

In [ ]:
# YOUR CODE HERE: Prepare test data

# Step 1: Define columns list
# columns = ['avg_energy', 'std_sensor_A', ...]

# Step 2: Load sample data
# Hint: dataset = spark.table('turbine_hourly_features').select(*columns).toPandas()[:5]

# Step 3: Display data
# print(dataset)

In [ ]:
# YOUR CODE HERE: Call API and get predictions
# Hint: predictions = score_model(dataset)
#       print(predictions)

### ✅ Success Criteria:
- API call successful (status 200)
- Predictions returned for all 5 rows
- Predictions are numeric (0, 1, 2, ...)
- Response time < 5 seconds

---

## 📊 Task 11: Compare Batch vs Real-Time Performance

**What you need to do:**
1. Measure API response time for 1 row
2. Measure API response time for 100 rows
3. Compare with batch UDF performance
4. Analyze when to use each approach

**💡 Use Python's `time` module to measure duration**

In [ ]:
# YOUR CODE HERE: Test API with 1 row

# import time
# 
# single_row = dataset.iloc[:1]
# 
# start = time.time()
# prediction = score_model(single_row)
# duration = time.time() - start
# 
# print(f"⏱️ API latency (1 row): {duration:.3f} seconds")
# print(f"Prediction: {prediction}")

In [ ]:
# YOUR CODE HERE: Test API with 100 rows

# large_dataset = spark.table('turbine_hourly_features').select(*columns).toPandas()[:100]
# 
# start = time.time()
# predictions = score_model(large_dataset)
# duration = time.time() - start
# 
# print(f"⏱️ API latency (100 rows): {duration:.3f} seconds")
# print(f"Throughput: {100/duration:.1f} predictions/second")

### 🤔 Reflection Questions:

**1. How fast is the API for single predictions?**
```
[Your answer: Response time in seconds]
```

**2. How does throughput compare: API vs Batch UDF?**
```
[Your answer: Predictions per second for each method]
```

**3. When would you choose API over batch UDF?**
```
[Your answer: Consider latency, volume, use case]
```

**4. What happens if endpoint scales to zero and you send a request?**
```
[Your answer: Think about cold start time]
```

---

## 🎁 BONUS Task: SQL Testing with ai_query() (Optional)

**⚠️ Note:** `ai_query()` is NOT available in Free Edition, but we show it for learning purposes.

**What it does:**
- Call ML endpoints directly from SQL
- Apply batch predictions without Python
- Accessible to business analysts

**Try this (will fail in Free Edition):**

In [ ]:
# BONUS: Try ai_query (will fail in Free Edition)

%sql
-- This demonstrates ai_query syntax (requires paid edition)
-- SELECT ai_query(
--     'e2eai_iot_turbine_prediction_endpoint',
--     STRUCT(
--         CAST(avg_energy AS DOUBLE) AS avg_energy, 
--         CAST(std_sensor_A AS DOUBLE) AS std_sensor_A, 
--         CAST(std_sensor_B AS DOUBLE) AS std_sensor_B, 
--         CAST(std_sensor_C AS DOUBLE) AS std_sensor_C, 
--         CAST(std_sensor_D AS DOUBLE) AS std_sensor_D, 
--         CAST(std_sensor_E AS DOUBLE) AS std_sensor_E,
--         CAST(std_sensor_F AS DOUBLE) AS std_sensor_F
--     ), 
--     returnType => 'FLOAT'
-- ) AS prediction
-- FROM turbine_hourly_features
-- LIMIT 3

---

## 🎉 Congratulations!

You've completed the Model Deployment Challenge! 🏆

### What You've Accomplished:

✅ **Batch Inference** - Spark UDF for large-scale predictions (Level 1)  
✅ **SQL Integration** - ML predictions in SQL queries  
✅ **REST API Deployment** - Production serving endpoint (Level 2)  
✅ **API Client** - Robust function for endpoint calls  
✅ **Performance Testing** - Compared batch vs real-time (Level 3)  
✅ **MLOps Best Practices** - Versioning, scale-to-zero, monitoring  

### Deployment Summary:

**Batch Inference (Spark UDF):**
- ✅ UDF created and registered
- ✅ Predictions saved in `turbine_hourly_predictions`
- ✅ Available in SQL for analysts

**Real-Time API:**
- ✅ Serving endpoint deployed
- ✅ Scale-to-zero enabled
- ✅ API tested successfully
- ✅ Response time: [Your result] seconds

---

## 📈 Next Steps in Production:

**📊 Monitoring:**
- Track prediction accuracy over time
- Monitor API latency and errors
- Set up alerts for model drift

**🔄 Model Updates:**
- Retrain model with new data
- Use A/B testing for new versions
- Roll back if performance degrades

**🔐 Security:**
- Implement role-based access control
- Audit endpoint usage
- Encrypt predictions at rest

**💰 Cost Optimization:**
- Monitor endpoint costs
- Adjust scale-to-zero timeout
- Use batch inference for scheduled jobs

---

## 💭 Final Reflection

**1. Which deployment strategy is best for your use case?**
```
[Your answer: Batch UDF vs REST API - justify your choice]
```

**2. What was the most challenging part of deployment?**
```
[Your answer]
```

**3. How would you monitor this model in production?**
```
[Your answer: Think about metrics, alerts, dashboards]
```

**4. What improvements would you make for a real production system?**
Options:
- Load balancing for high traffic
- Caching for frequently requested predictions
- Automatic retraining pipeline
- Feature store integration
- A/B testing framework
```
[Your answer]
```

**5. Explain to a business stakeholder: Why did we deploy the model two ways?**
```
[Your answer: Focus on business value - when to use batch vs real-time]
```

---

## 🌟 Key Takeaways

**MLOps is about more than just models:**
- ✅ Lineage - Track what data created what model
- ✅ Repeatability - Consistent environments and results
- ✅ Consumption - Make predictions accessible
- ✅ Performance - Monitor and improve over time

**Two deployment patterns serve different needs:**
- **Batch (UDF):** High volume, scheduled, cost-effective
- **Real-Time (API):** Low latency, on-demand, flexible

**Production readiness requires:**
- Versioning and governance (Unity Catalog)
- Error handling and retries
- Monitoring and alerting
- Cost optimization (scale-to-zero)
- Security and access control

---

**🎓 You're now ready for production ML deployments!** 🚀

**Want to see your endpoint in action?**
Go to **Serving** in the Databricks UI → Click your endpoint → View usage statistics and code examples!